In [1]:
!pip install hyperopt

In [14]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
ERROR! Session/line number was not unique in database. History logging moved to new session 61


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']

X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train, 
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
    )
  return model

def predict(model_trained, X_test, y_test, scoring= accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return accuracy_score(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),


  Dense(num_classes, activation='softmax')
    ])

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.3763 - accuracy: 0.3316 - val_loss: 0.7427 - val_accuracy: 0.7662
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.6423 - accuracy: 0.8011 - val_loss: 0.1767 - val_accuracy: 0.9511
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.2936 - accuracy: 0.9114 - val_loss: 0.0546 - val_accuracy: 0.9853
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2073 - accuracy: 0.9384 - val_loss: 0.0410 - val_accuracy: 0.9879
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1627 - accuracy: 0.9530 - val_loss: 0.0328 - val_accuracy: 0.9902


0.9798185941043084

In [12]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 4ms/step - loss: 0.0988 - accuracy: 0.9798


[0.0987817794084549, 0.97981858253479]

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax')
    ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train, 
      batch_size= int(params.get('batch_size', 128)) ,
      epochs=5,
      verbose=0,
    )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy= score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [34]:
space = {
  'batch_size': hp.quniform('batch_size', 100, 200, 10), 
  'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.7), 
  'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.7),
  'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.7),
  
  'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
  'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.623371364983407, 'dropout_cnn_block_three': 0.3946031424499532, 'dropout_cnn_block_two': 0.506899265079797, 'dropout_dense_block_one': 0.34464123161803023, 'dropout_dense_block_two': 0.5981089118582191}
accuracy=0.6167800426483154
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.5874519689054156, 'dropout_cnn_block_three': 0.5646212808442239, 'dropout_cnn_block_two': 0.5963108947927344, 'dropout_dense_block_one': 0.4229807162754502, 'dropout_dense_block_two': 0.31886007289699125}
accuracy=0.655102014541626
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.34105958913062373, 'dropout_cnn_block_three': 0.487476419940714, 'dropout_cnn_block_two': 0.5440746001109815, 'dropout_dense_block_one': 0.5530485096083644, 'dropout_dense_block_two': 0.6466798017405866}
accuracy=0.7376417517662048
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4729094022550696, 'dropout_cnn_block_three': 0.3986143310980474, 'dropout_cnn_block_two': 0.5144101302086297,